In [1]:
class TakeawayPlayer:
    def __init__(self, player_name, strategy):
        """
        Instantiates a new TakeawayPlayer object.
        
        Parameters:
            player_name (string): Name of the player
            strategy (function): Function to determine what move to make
        
        Returns:
            A newly instatiated TakeawayPlayer
        """
        self.player_name = player_name
        self.strategy = strategy

    def move(self, board) -> int:
        """
        Makes a move based on the player's strategy.
        
        Parameters:
            board (TakeawayBoard): The board being played on
        
        Returns:
            A move based on the player's strategy
        """
        return self.strategy.move(board)

In [2]:
def run_game(num_games, toothpicks, p1_strat_name, p2_strat_name, display_output = False):
    """
    Runs a specified number of games.
    
    Parameters:
        num_games (int): Number of games to run
        toothpicks (int): Initial number of toothpicks to use
        p1_strat_name (string): Name of strategy for player 1 to use
        p2_strat_name (string): Name of strategy for player 2 to use
        display_output (bool): Whether to display output to the console
    """
    # Obtain the strategies for each player
    p1_strategy = TakeawayStrategy(p1_strat_name)
    p2_strategy = TakeawayStrategy(p2_strat_name)

    # We're going to collect a lot of data
    game_data = []

    # Create a board of toothpicks
    board = TakeawayBoard(toothpicks)
    # Create a referee to govern the board
    referee = TakeawayReferee(board)

    # Create two players; each with a unique strategy
    player_1 = TakeawayPlayer(player_name = "player_1", strategy = p1_strategy)
    player_2 = TakeawayPlayer(player_name = "player_2", strategy = p2_strategy)

    for _ in range(num_games):
        # Create the game
        game = TakeawayGame(board, referee, [player_1, player_2])

        # Play the game
        summary = game.play(narrated = False)
        game_data.append(summary)

    if display_output:
        for summary in game_data:
            print()
            for turn in summary.history:
                print(turn, summary.history[turn])
            print("WINNER: ", summary.winner)

    # Write to csv
    filename = "{}_{}_{}.csv".format(p1_strat_name, p2_strat_name, num_games)
    write_to_csv(game_data, filename)

In [3]:
class TakeawayBoard:
    def __init__(self, num_toothpicks = 10):
        """
        Instantiates a new TakeawayBoard object.
        
        Parameters:
            num_toothpicks (int): The number of toothpicks on the board
        
        Returns:
            A newly instatiated TakeawayBoard
        """
        self.state = num_toothpicks
        self.initial_size = num_toothpicks
        self.summary = TakeawayData()

    def possible_moves(self) -> list:
        """
        Returns a list of all possible moves.
        
        Returns:
            A list representing all possible moves at the current state of the board
        """
        return [1, 2] if self.state > 1 else [1]

    def reset(self):
        """
        Resets the game board.
        """
        self.state = self.initial_size
        self.summary = TakeawayData()

In [4]:
class TakeawayReferee:
    def __init__(self, board):
        """
        Instantiates a new TakeawayReferee object.
        
        Parameters:
            board (TakeawayBoard): The board being played on
        
        Returns:
            A newly instatiated TakeawayReferee
        """
        self.is_game_over = False
        self.board = board

    def update(self, player) -> int:
        """
        Executes a 'turn' of the game by asking for a move, applying that move, and updating the game history.
        
        Parameters:
            player (TakeawayPlayer): The player making the current move
        
        Returns:
            The move made at the current turn
        """
        # Get current (legal) move
        current_move = self.ask_for_move(player)
        
        # Record the move history
        self.board.summary.record_move(self.board.state, player, current_move)
        
        # Update game board
        self.board.state -= current_move

        return current_move

    def ask_for_move(self, player) -> int:
        """
        Requests a move from the current player.
        
        Parameters:
            player (TakeawayPlayer): The player making the current move
        
        Returns:
            The proposed move to make
        """
        # To check for repeats
        deciding = True

        while deciding:
            # See what move the player would want to make.
            proposed_move = player.move(self.board)

            # Check to see if player has given up
            if proposed_move == 0 or proposed_move is None:
                deciding = False
                proposed_move = 0

            # If the move was legal, exit the loop
            elif self.is_legal(proposed_move):
                deciding = False

        return proposed_move

    def is_legal(self, move) -> bool:
        """
        Determines if a move is legal to make.
        
        Parameters:
            move (int): The proposed move to check the legality of
        
        Returns:
            True if the move is legal, else false
        """
        return ((move > 0) and (move < 3) and (self.board.state - move >= 0))
    
    def check_for_winner(self, move, player, opponent) -> TakeawayPlayer:
        """
        Checks to see if someone has won the game.
        
        Parameters:
            move (int): The proposed move to check the legality of
            player (TakeawayPlayer): The player making the current move
            opponent (TakeawayPlayer): The current opponent of the turn
        
        Returns:
            The winning player, if applicable, else None
        """
        winner = None
        if move == 0 or move is None:
            winner = opponent
        elif self.board.state == 0:
            winner = player
        
        # Only assign winner if winner was found
        if winner is not None:
            self.board.summary.winner = winner.player_name

        return winner

In [5]:
class TakeawayGame:
    def __init__(self, board, referee, players):
        """
        Instantiates a new TakeawayGame object.
        
        Parameters:
            board (TakeawayBoard): The board being played on
            referee (TakeawayReferee): The referee for the game
            players (list): The players playing the game
        
        Returns:
            A newly instatiated TakeawayGame
        """
        self.referee = referee
        self.board = board
        self.players = players

    def play(self,narrated = False) -> (TakeawayPlayer, dict):
        """
        Plays a single game of Toothpick Takeaway.
        
        Parameters:
            narrated (bool): Whether to narrate the game
        
        Returns:
            The winner of the game and the move history
        """
        self.board.reset()
        turn = 0
        winner = None

        while winner is None:
            player = self.players[turn % 2]
            opponent = self.players[(turn + 1) % 2]
            turn = turn + 1

            move = self.referee.update(player)
            winner = self.referee.check_for_winner(move, player, opponent)

            if (narrated):
                print(player.player_name, "drew", move, "toothpicks.", self.board.state, "left")
                
        return self.board.summary

In [6]:
from random import choice

class TakeawayStrategy:
    def __init__(self, name = "random", data = None, bias = None):
        """
        Instantiates a new TakeawayStrategy object.
        
        Parameters:
            name (string): The name of the strategy to use
            data (DataFrame): Move data
            bias (int): What move the strategy defaults to, if any
        
        Returns:
            A newly instatiated TakeawayStrategy
        """
        self.name = name
        self.data = data
        self.bias = bias
        self.strategies = {"random": self.random_move,
                           "take_one": self.always_take_one,
                           "take_two": self.always_take_two,
                           "smart": self.smart_move,
                           "human": self.human
                          }
        
    def move(self, board) -> int:
        """
        Makes a move based on the current strategy.
        
        Parameters:
            board (TakeawayBoard): The board being played on
        
        Returns:
            A move to try
        """
        deciding = True
        possible_moves = board.possible_moves()
        moves_tried = []
        attempts = 1
        move_to_try = 0
        while deciding:
            # Get a move
            move_to_try = self.strategies[self.name](board, possible_moves)
            
            # If the move is invalid, note it and re-loop
            # Otherwise, end the loop
            if (board.state - move_to_try) < 0:
                moves_tried.append(move_to_try)
                attempts += 1
            else:
                deciding = False
            
            # If all possible moves have been tried or 3 attempts have been made
            if set(moves_tried) == set(possible_moves) or attempts == 3:
                deciding = False
                move_to_try = 0

        return move_to_try
    
    def smart_move(self, board, possible_moves) -> int:
        """
        Moves intelligently based on previous game data.
        
        Parameters:
            board (TakeawayBoard): The board being played on
            possible_moves (list): All possible moves on the current state of the board
        
        Returns:
            A move to try
        """
        # Get the otpimal move
        move_to_try = 1 if self.data["Take 1 Win %"][board.state] > self.data["Take 2 Win %"][board.state] else 2
        
        # If both chances are equal, choose randomly or according to a bias, if supplied
        if self.data["Take 1 Win %"][board.state] == self.data["Take 2 Win %"][board.state]:
            move_to_try = choice(possible_moves) if self.bias is None else self.bias

        # If the move is not legal, mark it as tried and pick the only other option
        if (board.state - move_to_try) < 0:
            possible_moves.remove(move_to_try)
            move_to_try = possible_moves[0]

        # If the only other move available is invalid, return None
        if (board.state - move_to_try) < 0:
            return 0

        return move_to_try

    def random_move(self, board, possible_moves) -> int:
        """
        Moves randomly based on available moves.
        
        Parameters:
            board (TakeawayBoard): The board being played on
            possible_moves (list): All possible moves on the current state of the board
        
        Returns:
            A move to try
        """
        return choice(possible_moves)

    def always_take_one(self, board, possible_moves) -> int:
        """
        Always takes 1 toothpick.
        
        Parameters:
            board (TakeawayBoard): The board being played on
            possible_moves (list): All possible moves on the current state of the board
        
        Returns:
            1, to take 1 toothpick
        """
        return 1

    def always_take_two(self, board, possible_moves) -> int:
        """
        Always takes 2 toothpicks.
        
        Parameters:
            board (TakeawayBoard): The board being played on
            possible_moves (list): All possible moves on the current state of the board
        
        Returns:
            2, to take 2 toothpicks
        """
        return 2

    def human(self, board, possible_moves) -> int:
        """
        Requests human input to decide a move.
        
        Parameters:
            board (TakeawayBoard): The board being played on
            possible_moves (list): All possible moves on the current state of the board
        
        Returns:
            A move to try
        """
        return int(input("Please make your move > "))

In [7]:
class TakeawayData:
    def __init__(self):
        """
        Instantiates a new TakeawayData object.
        
        Returns:
            A newly instantiated TakeawayData
        """
        self.history = {}
        self.winner = None
        
    def record_move(self, toothpicks_left, player, move):
        """
        Inserts a given move into the game data.
        
        Parameters:
            toothpicks_left (int): Number of toothpicks left on the table
            player (TakeawayPlayer): The player making the move
            move (int): The move being made
        """
        self.history[toothpicks_left] = {"name": player.player_name, "move": move}
            
    def reset(self):
        """
        Resets the game data.
        """
        self.history = {}
        self.winner = None

In [10]:
def write_to_csv(game_data, filename):
    """
    Writes game data to a .csv file.
    
    Parameters:
        game_data (list): Data of each turn made in the game and the winner of that game
        filename (string): Name of the file to write to
    """
    # Get the total number of toothpicks at the start
    start_val = max(game_data[0].history.keys())
    
    # Make a descending list of all toothpicks left
    toothpicks_left = list(range(start_val, 0, -1))

    # Create our headings: Toothpicks left and turn
    headings = []
    for heading in toothpicks_left:
        headings.append(heading)
        headings.append("turn_{}".format(heading))
    headings.append("winner")

    # Start building rows; one row per game
    rows = []
    for summary in game_data:
        # How many toothpicks were taken at each turn in the game
        turns = [summary.history[turn]["move"] for turn in summary.history]
        # Who took those toothpicks
        names = [summary.history[turn]["name"] for turn in summary.history]
        
        # Start creating a row
        moves = []
        for i in range(len(turns)):
            # Append the toothpicks taken
            moves.append(turns[i])
            # Append the player who took them
            moves.append(names[i])

            # If a turn was 2, add a turn of 0 after it
            # This ensures that our rows are all the same length
            if turns[i] == 2:
                moves.append(None)
                moves.append(None)

        # Append the winner of the game
        moves.append(summary.winner)

        # Add the row we just made to the running list of rows
        rows.append(moves)

    # Write to csv
    import csv
    with open(filename, "w") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(headings)
        csvwriter.writerows(rows)

In [11]:
run_game(10, 10, "random", "random")